In [ ]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
# Read in the CSV file (data.csv) as a DataFrame
ufc_df = pd.read_csv("Resources/data.csv")
ufc_df.head()

In [ ]:
# Drop the non-beneficial columns
ufc_df = ufc_df.drop(
    columns=[
        "BPrev",
        "RPrev",
        "BStreak",
        "B_Location",
        "R_Location",
        "Event_ID",
        "Fight_ID",
        "B_ID",
        "R_ID",
        "B_HomeTown",
        "R_HomeTown",
        "Date",
    ]
)
ufc_df.head()

In [ ]:
# Keep only wins and losses (i.e., Red & Blue)

# Display `value_counts()` on `winner` column before modification
print("Before", "-" * 20, ufc_df.winner.value_counts(), "-" * 20, "\n", sep=os.linesep)

ufc_df = ufc_df.loc[(ufc_df.winner == "blue") | (ufc_df.winner == "red")]

# Display results
print("After", "-" * 20, ufc_df.winner.value_counts(), "-" * 20, sep=os.linesep)


## AutoML To Select Best Classifier

In [ ]:
import numpy as np
from tpot import TPOTClassifier
from tpot import TPOTRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# ufc_subset_df = ufc_df.convert_dtypes()

numeric_features = [
    "Max_round",
    "B_Age",
    "B_Height",
    "B_Weight",
    "R_Age",
    "R_Height",
    "R_Weight",
]
categorical_features = ["winby"]
target_value = ["winner"]
subset_features = numeric_features + categorical_features + target_value

# Conduct analysis only on subset of features
ufc_subset_df = ufc_df[subset_features]

# Drop rows that contain null values
# ufc_subset_df.isnull().sum()
ufc_subset_df = ufc_subset_df.dropna(axis=0, how="any")

# Encode categorical features with `get_dummies``
ufc_subset_df = pd.get_dummies(ufc_subset_df, columns=categorical_features)

# "Encode" text as numbers 
ufc_subset_df["winner"] = np.where(
    ufc_subset_df["winner"] == "red", 0, 1
)  # 0 = value if true (red), 1 = value if not true (blue)

# Reset index
ufc_subset_df = ufc_subset_df.reset_index(drop=True)

# Set features (X) and target (y)
X = ufc_subset_df.drop("winner", axis=1)
y = ufc_subset_df["winner"]

# Split data into test/train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Run tpot classifier on light mode
tpot = TPOTClassifier(
    generations=5, population_size=20, verbosity=2, config_dict="TPOT light"
)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

## Logistic Regression

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.compose import make_column_selector as selector
from sklearn.linear_model import LogisticRegression

# Imputation transformer for completing missing values.
# Standardize features by removing the mean and scaling to unit variance with `StandardScalar()`.
numeric_transformer = Pipeline(
    # steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
    steps=[("imputer", SimpleImputer(strategy="constant")), ("scaler", StandardScaler())]
)

categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="object")),
        ("cat", categorical_transformer, selector(dtype_include="object")),
    ]
)

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression(C=0.5, dual=False, penalty="l2")),
    ]
)

X = ufc_subset_df.drop("winner", axis=1)
y = ufc_subset_df["winner"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

### Display Diagram of Pipeline

In [ ]:
from sklearn import set_config

set_config(display="diagram")
clf

### Classification Report

In [ ]:
from sklearn.metrics import classification_report

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

## Regression without ColumnTransformer

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Split features and target arrays

# Binary encoding
dummies_df = pd.get_dummies(ufc_df)
X = dummies_df.drop(columns=["winner_blue", "winner_red"])
X = X.fillna(0)

y = ufc_df["winner"]
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=1
)

# Scaling the data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create and train the Logistic Regression Model
clf = LogisticRegression(solver="lbfgs", max_iter=200)
clf.fit(X_train_scaled, y_train)

# Predict Outcomes
y_pred = clf.predict(X_test_scaled)

# Print accuracy score
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

## Test: Regression with Subset of Features

In [ ]:
numeric_features = ["Max_round", "B_Age", "B_Height", "B_Weight", "R_Age", "R_Height", "B_Weight"]
categorical_features = ["winby"]
target_value = ["winner"]

subset_features = numeric_features + categorical_features + target_value

# Select subset of features
ufc_subset_df = ufc_df[subset_features]

# Drop null (?)
# ufc_subset_df = ufc_subset_df.dropna(how="any", axis="rows")

# Reset index
ufc_subset_df = ufc_subset_df.reset_index(drop=True)

In [ ]:
# Binary encoding
dummies_df = pd.get_dummies(ufc_subset_df)
X = dummies_df.drop(columns=["winner_blue", "winner_red"])
X = X.fillna(0)

y = ufc_subset_df["winner"]
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=1
)

# Scaling the data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create and train the Logistic Regression Model
clf = LogisticRegression(solver="lbfgs", max_iter=200)
clf.fit(X_train_scaled, y_train)

# Predict Outcomes
y_pred = clf.predict(X_test_scaled)

# Print accuracy score
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

## Test: StandardScaler on get_dummies()

In [ ]:
# TODO: Should StandScalar() be applied to non-numerical categories that got encoded? (Ex. OneHotEncoder() or get_dummies()) 

# We know that there are 4 columns that are going to be encoded with get_dummies() since they are `object` dtype
# `columns=` -> Column names in the DataFrame to be encoded. If columns is None then all the columns with object or category dtype will be converted.
display(ufc_df.dtypes.value_counts())

# Should `dummy_na=True`? -> Add a column to indicate NaNs, if False NaNs are ignored.
display(ufc_df.select_dtypes(include="object").isnull().sum())


# We are encoding the object variables into uint8
dummies_df = pd.get_dummies(ufc_df)
display(dummies_df.dtypes.value_counts())

# Should StandScalar() be applied to non-numerical categories that got encoded? (Ex. OneHotEncoder() or get_dummies()) 
# dummies_df.select_dtypes(include="uint8").columns.tolist()
# dummies_df.select_dtypes(exclude="uint8").columns.tolist()

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html